# Import Libraries 

In [7]:
import tensorflow as tf
from tensorflow import keras

# `image_dataset_from_directory` Function and its usage 
 `image_dataset_from_directory`is a powerful function that can convert a whole folder of images into a dataset by using just some tweaks


In [ ]:
from tensorflow.keras.preprocessing import image_dataset_from_directory

# Define dataset directory
dataset_dir="E:\Facial_Expression_Dataset"

train_ds = image_dataset_from_directory(
    dataset_dir,
    validation_split=0.2,  # 20% for validation/test
    subset="training",
    seed=123, 
    image_size=(150, 150),  
    labels="inferred",
    label_mode="int",
    batch_size=32
)

val_ds = image_dataset_from_directory(
    dataset_dir,
    validation_split=0.2,
    subset="validation",
    seed=123,
    image_size=(150, 150),
    labels="inferred",
    label_mode="int",
    batch_size=32
)

class_names = train_ds.class_names
print("Class Names:", class_names)



Found 7674 files belonging to 2 classes.
Using 6140 files for training.
Found 7674 files belonging to 2 classes.
Using 1534 files for validation.
Class Names: ['Happy', 'Sad']


-`dataset_dir`: Path to the main folder containing class subfolders (e.g., cats, dogs).

-`validation_split`=0.2: Reserves 20% of the data for validation.

-`subset`="training": Specifies that this dataset is the training part of the split.

-`seed`=123: Sets a fixed seed to ensure the split is consistent on each run.

-`image_size`=(150, 150): Resizes all input images to 150x150 pixels.

-`labels`="inferred": Automatically assigns labels based on folder names.

-`label_mode`="int": Returns labels as integers like 0, 1, 2 (not one-hot encoded). Other options for label mode: "categorical","binary"

-`batch_size`=32: Loads 32 images at a time during training.

# Normalize image for scaling data

In [9]:
def normalize(image,label):
    image=tf.cast(image/255,tf.float32)
    return image,label

In [10]:
train_ds=train_ds.map(normalize)
val_ds=val_ds.map(normalize)

# Write a very LeNet-5 Architecture for training

In [ ]:
from tensorflow.keras import layers

model = keras.Sequential([
    layers.Conv2D(32, (3,3), activation='relu', input_shape=(150, 150, 3)),  # Ensure this matches
    layers.MaxPooling2D(2,2),

    layers.Conv2D(64, (3,3), activation='relu'),
    layers.MaxPooling2D(2,2),

    layers.Conv2D(128, (3,3), activation='relu'),
    layers.MaxPooling2D(2,2),

    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(2, activation='softmax')  
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',  # Use this for integer labels
              metrics=['accuracy'])


In [12]:
history=model.fit(train_ds,epochs=7,validation_data=val_ds)

Epoch 1/7
192/192 ━━━━━━━━━━━━━━━━━━━━ 75s 383ms/step - accuracy: 0.5711 - loss: 0.7214 - val_accuracy: 0.7986 - val_loss: 0.4439
Epoch 2/7
192/192 ━━━━━━━━━━━━━━━━━━━━ 70s 365ms/step - accuracy: 0.8127 - loss: 0.4226 - val_accuracy: 0.8364 - val_loss: 0.3904
Epoch 3/7
192/192 ━━━━━━━━━━━━━━━━━━━━ 147s 767ms/step - accuracy: 0.8619 - loss: 0.3238 - val_accuracy: 0.8735 - val_loss: 0.3176
Epoch 4/7
192/192 ━━━━━━━━━━━━━━━━━━━━ 152s 789ms/step - accuracy: 0.8940 - loss: 0.2582 - val_accuracy: 0.8827 - val_loss: 0.2972
Epoch 5/7
192/192 ━━━━━━━━━━━━━━━━━━━━ 197s 765ms/step - accuracy: 0.9167 - loss: 0.2021 - val_accuracy: 0.8872 - val_loss: 0.3105
Epoch 6/7
192/192 ━━━━━━━━━━━━━━━━━━━━ 172s 889ms/step - accuracy: 0.9340 - loss: 0.1683 - val_accuracy: 0.8787 - val_loss: 0.3323
Epoch 7/7
192/192 ━━━━━━━━━━━━━━━━━━━━ 129s 671ms/step - accuracy: 0.9486 - loss: 0.1361 - val_accuracy: 0.8937 - val_loss: 0.3292


# Save the entire model (architecture + weights + optimizer state)

In [ ]:

model.save("model.keras")  # Save it as an HDF5 file

# Let's test our model for a demo image

In [ ]:
from tensorflow.keras.preprocessing import image
import numpy as np
import matplotlib.pyplot as plt
import cv2

# Write the preprocess function

In [ ]:

class_names = train_ds.class_names # ["Happy", "Sad"]

# Step 1: Load the saved model
model=tf.keras.models.load_model('E:\Machine Learning\Image Classification\Facial Expression Detection Project\models\model.keras')

# Step 2: Preprocess the image
def preprocess_image(image_path):
    img = image.load_img(image_path, target_size=(150, 150))
    
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)  
    img_array = img_array / 255.0 # Normalize the image to [0, 1]
    
    return img_array


# Test on demo image

In [ ]:
image_path = r"E:\Machine Learning\Image Classification\image.png"

preprocessed_image = preprocess_image(image_path)

predictions = model.predict(preprocessed_image)

# Get the index of the predicted class (class with the highest probability)
predicted_class_index = np.argmax(predictions, axis=1)
# Map the index to the actual class name
predicted_class = class_names[predicted_class_index[0]]
print(f"Predicted Class: {predicted_class}")